In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ============================ read data ======================
attlist = ['latitude', 'longitude', 'last_days', 'pvSizeWatt']
# dataset with the recorded data time periods of each pv system
df_lasttime = pd.read_csv('../preprocessed_data/daily_last_time_20231016.csv', header=0)
df_lasttime['last_days'] = df_lasttime['last_days'].str.split(' ').str[0]

# table with timezone
df_tz = pd.read_csv('../input_data/SITE_nodeType_20230630.csv', header=0) 

# table with latitude & longitude & pvsize
df_att = pd.read_csv('../input_data/MNTR_ddb_20230630.csv', header=0)
df_att['timezone'] = df_att['siteId'].map(df_tz.set_index('source')['timezone'])
df_att['last_days'] = df_att['source'].map(df_lasttime.set_index('source')['last_days'])
df_att['last_days'] = df_att['last_days'].replace(np.nan, 0)
df_att['latitude'] = [x[1:] for x in df_att['latitude']]

for att in attlist:
    df_att[att] = df_att[att].astype(float)
print(len(df_att))
df_att.head()

In [ ]:
import folium
m1 = folium.Map([-34.69016, 138.68792], zoom_start=4)
for index, row in df_att.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                popup=row['pvSizeWatt'],
                # tooltip=row[''],              
                ).add_to(m1)
m1

In [ ]:
from folium import plugins
import time
m2 = folium.Map([-35.69016, 138.68792], zoom_start=4, width="%100",height="%100")
locations = list(zip(df_att.latitude, df_att.longitude))
cluster = plugins.MarkerCluster(locations=locations,                     
               popups=df_att["source"].tolist())  
m2.add_child(cluster)
heat_data = df_att.groupby(["latitude","longitude"])['pvSizeWatt'].mean().reset_index().values.tolist()

folium.plugins.HeatMap(heat_data).add_to(m2)

# # save 
# m2.save('output.html')
m2

In [6]:
import branca.colormap as cm
pvSize_max = df_att['pvSizeWatt'].max()/1000
pvSize_min = df_att['pvSizeWatt'].min()/1000


In [ ]:

m3 = folium.Map([-34.69016, 138.68792], zoom_start=4, width="%100", height="%100")

lastday_min = df_att['last_days'].min()
lastday_max = df_att['last_days'].max()

colormap = cm.StepColormap(colors=['limegreen','red','blue','darkorange', 'darkviolet', 'black'] ,#renkler
                           vmin= lastday_min, vmax= lastday_max, 
                           index = [lastday_min, 30, 90, 180, 300, 365, lastday_max],
                           caption='Recorded Time Peroids of PV Systems (day), Circle size represents the PV size')
colormap.to_linear()

for index, row in df_att.iterrows():
    pVsizer = row['pvSizeWatt']/1000
    last_timer = row['last_days']
    folium.Circle(
          location=[row['latitude'], row['longitude']],
          popup= 'pvsize:' + str(pVsizer) + ' ' + row['source'] + '||last_days=' + str(row['last_days']),
          radius=row['pvSizeWatt']/8,
          color=colormap(last_timer),
          fill=True,
          fill_color=colormap(last_timer),
       ).add_to(m3)
colormap.add_to(m3)
m3


In [ ]:
m4 = folium.Map([-34.69016, 138.68792], zoom_start=4, width="%100", height="%100")
colormap = cm.StepColormap(colors=['green','red','blue','orange', 'purple', 'black'] ,#renkler
                           vmin= pvSize_min, vmax= pvSize_max, 
                           index = [pvSize_min, 100, 300, 1000, 1500, 2000, pvSize_max],
                           caption='PV Size of Each PV System (kW), Circle Size Represents the Recorded Time Period')
colormap.to_linear()
for index, row in df_att.iterrows():
    pVsizer = row['pvSizeWatt']/1000
    if row['last_days'] is None:
        row['last_days'] = 0
    else:
        row['last_days'] = float(row['last_days'])
    folium.Circle(
          location=[row['latitude'], row['longitude']],
          popup= 'pvsize:' + str(pVsizer) + ' ' + row['source'] + '||last_days=' + str(row['last_days']),
          radius=row['last_days']*200,
          color=colormap(pVsizer),
          fill=True,
          fill_color=colormap(pVsizer),
       ).add_to(m4)
colormap.add_to(m4)
# m4.save('pvsize_kw_map.html')
m4